In [41]:
%pip install -q openai langchain langchain_community pypdf chromadb langchain-openai gradio

Note: you may need to restart the kernel to use updated packages.


In [33]:
import os
from dotenv import load_dotenv

load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found in environment variables")

In [34]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader = PyPDFLoader("pdf/1728286846_the_nestle_hr_policy_pdf_2012.pdf")
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=150)

chunks: list[Document] = loader.load_and_split(text_splitter)


In [35]:
from langchain_community.vectorstores import Chroma
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=openai_api_key, model="text-embedding-3-small")
vector_store = Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory="chroma_nestle")

In [ ]:
from langchain_openai import ChatOpenAI

retriever = vector_store.as_retriever(search_kwargs={"k":3})

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are Nestlé’s HR assistant..."),
    ("human", "Context:\n{context}\n\nQuestion: {question}")
])

chain = (
    {"question": RunnablePassthrough(), 
     "context": retriever | (lambda docs: "\n\n".join(d.page_content for d in docs))}
    | prompt
    | ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
)

In [ ]:
import gradio as gr

def chat_handler(message, history):
    response = chain.invoke(message)
    return response.content

interface = gr.ChatInterface(
    fn=chat_handler,
)

interface.launch()

/Users/kevinluo/opt/anaconda3/envs/purdue-ai/lib/python3.11/site-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.
